In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### Transaction Dimension

In [6]:
%%time
# Data ingestion, cleaning/wrangling
df_td_raw = pull_data(config_path, td_sql(extract_end_date), 'DWH', 'td', False, 'raw')

Currently loading td data set ...


,mifos_loan_id,is_reversed,transaction_type_enum,transaction_date,bloom_version
123676,116833,False,2,2022-06-07,2
463085,160219,False,2,2019-11-18,1


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 422 ms
Wall time: 988 ms


In [7]:
%%time
# Data ingestion, cleaning/wrangling
df_td_clean = clean_dataset(config_path, 'td')


Currently cleaning td data set ...


,mifos_loan_id,is_reversed,transaction_type_enum,transaction_date,bloom_version
159829,49261,False,2,2022-06-23,2.00
529641,181981,False,2,2020-05-13,1.00


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 812 ms
Wall time: 2.2 s


In [8]:
%%time
# Feature engineering
df_td_features = td_feature_engineering(config_path)

,mifos_loan_id,is_reversed,transaction_type_enum,transaction_date,bloom_version,loan_id_product_concat
571049,201873,False,2,2021-02-15,1.00,201873-1.0
214044,149990,False,2,2022-07-14,2.00,149990-2.0


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 1.91 s
Wall time: 3.61 s


In [9]:
%%time
# Data aggregation
df_td_aggregate = td_agg_summaries(config_path)

,loan_id_product_concat,max_transaction_date
337647,74128-2.0,2022-04-23
382418,9857-2.0,2022-01-04


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 1.66 s
Wall time: 3.26 s


In [10]:
# Final dataset schema
df_td_aggregate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384860 entries, 0 to 384859
Data columns (total 2 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   loan_id_product_concat  384860 non-null  object        
 1   max_transaction_date    384860 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 5.9+ MB
